In [1]:
# Test file load_data

from importlib import reload
import os
os.chdir("D:/amazon-reviews") # Run always on the root to avoid problems


import pandas as pd
from src.load_data import load_info, multiple_files #I mport funtion from file
from src.text.clean_text import clean
import src.text.lang_detection as lt
reload(lt)
import src.pipeline.ingestion as ingest
reload(ingest)
import src.pipeline.merge_parquet as mp
reload(mp)


<module 'src.pipeline.merge_parquet' from 'D:\\amazon-reviews\\src\\pipeline\\merge_parquet.py'>

In [6]:
# Test 1

df = load_info("data/raw/Industrial_and_Scientific.json.gz")

df.head()

Reading file: Industrial_and_Scientific.json.gz ...


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000020D14909190>>
Traceback (most recent call last):
  File "d:\amazon-reviews\env\Lib\site-packages\ipykernel\ipkernel.py", line 812, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1535, in enumerate
    def enumerate():
    
KeyboardInterrupt: 


 Loaded 1758333 rows and 12 columns.


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"01 23, 2013",A3FANY5GOT5X0W,0176496920,Kelly Keyser,"Arrived on time, in mint condition, great! I ...",Just as described!,1358899200,NaN,NaN,NaN
1,5,True,"11 5, 2012",AT6HRPPYOPHMB,0176496920,Michael C,This device was hard to find for my daughter's...,Great device,1352073600,NaN,NaN,NaN
2,4,True,"10 17, 2012",A4IX7B38LIN1E,0176496920,BH,Just a clicker nothing special. Was hoping it ...,Pretty Good,1350432000,NaN,NaN,NaN
3,5,True,"03 29, 2017",A12Q4LR8N17AOZ,0176496920,Waterfall3500,Great response card. Slow shipping but it work...,Thank you for the great product. Works. A++ Us...,1490745600,NaN,NaN,NaN
4,1,True,"03 21, 2017",A1GJXZZPOZ3OD9,0176496920,Amazon Customer,It only lasted for 3 days before it stopped wo...,One Star,1490054400,NaN,NaN,NaN


In [3]:
# Test 2

files = multiple_files()

print(files)

[WindowsPath('data/raw/Electronics.json.gz')]


In [7]:
# Test 3 - Try the first step of the pipeline ingestion READ THE FILES using funtions created in load_data

raw_files = ingest.list_raw_files()

2 files found in data/raw/
Electronics.json.gz
Industrial_and_Scientific.json.gz


In [8]:
# Test 4 -  try to read one chunk

if len(raw_files) > 0: # Verify raw_files is not empty 
    file_one = raw_files[0]
    print(f"Chunk test in file: {file_one.name}")

    for chunk in pd.read_json(file_one, lines=True, chunksize=1000):
        print(f"{len(chunk)} rows loaded for first chunk")
        print(chunk.head()) # Show first rows
        break  # Just to test with chunk #1
else:
    print("No files found in data/raw/")


Chunk test in file: Electronics.json.gz
1000 rows loaded for first chunk
   overall  verified   reviewTime      reviewerID      asin  \
0        5      True  07 17, 2002  A1N070NS9CJQ2I  60009810   
1        5     False   07 6, 2002  A3P0KRKOBQK1KN  60009810   
2        5     False   07 3, 2002  A192HO2ICJ75VU  60009810   
3        4     False  06 30, 2002  A2T278FKFL3BLT  60009810   
4        5     False  06 28, 2002  A2ZUXVTW8RXBXW  60009810   

                       style reviewerName  \
0  {'Format:': ' Hardcover'}   Teri Adams   
1  {'Format:': ' Hardcover'}     Willa C.   
2  {'Format:': ' Hardcover'}          Kit   
3  {'Format:': ' Hardcover'}       Andres   
4  {'Format:': ' Hardcover'}         John   

                                          reviewText  \
0  This was the first time I read Garcia-Aguilera...   
1  As with all of Ms. Garcia-Aguilera's books, I ...   
2  I've not read any of Ms Aguilera's works befor...   
3  This romance novel is right up there with the ... 

In [10]:
# Try 4 - Apply the cleaning module to the reviewText column

# Take the column review text from chunk
rText = chunk['reviewText']

# Apply cleaning

rText_cleaned = rText.apply(clean)

for org, cleaned in zip(rText.head(5), rText_cleaned.head(5)):

    print(f"Before: {org}    ")
    print (f"After: {cleaned}")
    print("\n")



Before: This was the first time I read Garcia-Aguilera.  I came upon the name of this book on Live with Regis and Kelly. This book was exactly what I was looking for ... it hit the spot.  I really enjoyed this book because it was well written. Once I started this book it kept me coming back for more. It had culture, family, friendship and romance. I was looking for a little more romance when I picked this book but in the end it turned out to be just right.  I love the main chartachter Margarita (aka Daisy). I've never been to Miami but the way Daisy told the story I certainly felt I'd been there.
Also after going through all of Daisy's perils ... I closed the book with a feeling I had grown emotionally as well.    
After: this was the first time i read garcia-aguilera i came upon the name of this book on live with regis and kelly this book was exactly what i was looking for it hit the spot i really enjoyed this book because it was well written once i started this book it kept me coming

In [17]:
# Test 5 - Try the langdetect module on the chunk

english_rows = rText_cleaned.apply(lt.is_english)

cont = 0 

for a in english_rows:
    if a == True:
        cont+=1

print(f"Original size: {rText_cleaned.size}\n")
print(f"English reviews: {cont}")

Original size: 1000

English reviews: 921


In [ ]:
# Test 6 - Try the chunk loading with the pipeline module check

ingest.processing_check()

4 files found. Ingestion started.

Processing file Electronics.json.gz

5000 rows loaded

Original rows: 5000
English reviews: 4640
Reviews not considered: 360


Processing file Industrial_and_Scientific.json.gz

5000 rows loaded

Original rows: 5000
English reviews: 4259
Reviews not considered: 741


Processing file Musical_Instruments.json.gz

5000 rows loaded

Original rows: 5000
English reviews: 4556
Reviews not considered: 444


Processing file Video_Games.json.gz

5000 rows loaded



In [4]:
# Try 7 - Try the actual pipeline with parquet file saving
ingest.save_to_parquet()

Processing started...

Processing file Electronics.json.gz

Electronics.json already processed. Moving to the next file.

Processing file Home_and_Kitchen.json.gz


Writing in data\processed\Home_and_Kitchen.json.temp.parquet started.

Processing file Industrial_and_Scientific.json.gz

Industrial_and_Scientific.json already processed. Moving to the next file.

Processing file Musical_Instruments.json.gz

Musical_Instruments.json already processed. Moving to the next file.

Processing file Video_Games.json.gz

Video_Games.json already processed. Moving to the next file.

File data\processed\Video_Games.json.parquet generated successfully.


In [5]:
# Try 8 - Test the merging function

mp.merge_par()

Loaded 5 processed entries from TXT file.
dataset_embedding.parquet already exists, checking if there is new files to append...

No files to add.
